## How to do some GIS using Geopandas: 
#### An introduction for novices, by a novice. 
***Purpose:*** To showcase a clean Jupyter notebook that outputs webmap-ready data (GeoJSON).  This notebook will incorporate Geopandas, and a variety of other Python libraries to import, wrangle, analyze, and export data within a typical data-viz and mapping workflow.

###### Created by Ritchie Katko (rakatk0@uky.edu) for [UKy Geography's New Maps Plus](http://newmapsplus.uky.edu/) MAP674 Fall 2019

##### Objective: 
This notebook will fulfill its purpose through the exploration of New York City Police Department's ["Stop, Question, Frisk" Data](https://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page) for the year 2016.  This data originates as a CSV, projected in [EPSG:2263 NAD83 / New York Long Island (ftUS)](https://epsg.io/2263). It will have to be cleaned up and reprojected to be ready for analysis against other geospatial data (see below).

Please note the following: The data wrangling and analysis performed in this notebook is intended to function as a geopandas tutorial, rather than as a piece of data-journalism. 

Data utilized in this notebook: 
- [NYPD Precinct Boundaries](https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz)
- ["Stop, Question, Frisk" Data](https://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page)



#### Setting up the environment
Here's my background specs: 
- ***OS***: Ubuntu 16.04.6 LTS
- ***Python version***: `$ /usr/bin/python3 -V` 3.5.2
- ***Anaconda3 version***: `$ conda info` 4.5.11

###### Conda Environment 
The following packages must be installed (`$ conda install`) into the active Conda environment:
- `Jupyter` (allows use of jupyter coding environment
- `Geopandas` (integrates other packages geospatial components) 
- `Matplotlib` (dataviz package - allows plotting of visualizations)

##### after activating the Conda Environment of choice, navigate to the working directory and run `jupyter notebook` to initiate the browser-based notebook environment.  

##### Now, let's get to work: 

##### First we'll Add packages to allow for their use in this environment
From the [pandas website](https://pandas.pydata.org/): "pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language"

From the [numpy website](https://numpy.org/): "Besides its obvious scientific uses, NumPy can also be used as an efficient multi-dimensional container of generic data. Arbitrary data-types can be defined. This allows NumPy to seamlessly and speedily integrate with a wide variety of databases."

In [1]:
# import packages, using aliases to simplify code
import pandas as pd
import numpy as np

##### Load Local Data
We'll load a CSV into the notebook as dataframe `df`. This CSV is hosted locally within the repository. Using the `.type()` method, the cell will output the type of data.  In this case it is a `pandas.core.frame.DataFrame`

In [2]:
# load csv into notebook as dataframe 'df', and check the data  
path_to_file = './data/sqf-2016.csv'
# url version: path_to_file = 'https://www1.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/stop-question-frisk/sqf-2016.csv'

df = pd.read_csv(path_to_file,  dtype = str) # load csv data as pandas DataFrame, load # dtype = str, 
type(df)

pandas.core.frame.DataFrame

##### take a look at the data, from a few angles
Using `.shape` provides a simple look at the number of rows and columns.  using `.sample(X)` provides a random sampling of X # of rows for inspection.   `.head()` and `.tail()` return the first 5, and last 5 rows, respectively. Using `list(df.columns)` will return a list of all the columns within the dataframe. 

In [3]:
df.shape, df.sample(2)  # shape will return the number of rows, and columns.  .sample() will return a random row

((12405, 112),
       year  pct ser_num datestop timestop recstat inout trhsloc perobs  \
 9255  2016   81     107  8292016     2350       1     O       P   1.00   
 6609  2016  103      57  5262016     1550       1     O       P   1.00   
 
         crimsusp  ... zip addrpct sector beat post   xcoord   ycoord dettypCM  \
 9255         CPW  ...          81      B    4       1001222   189511       CM   
 6609  CPW- KNIFE  ...         103      B    3       1039709   196693       CM   
 
      lineCM detailCM  
 9255      1       20  
 6609      1       20  
 
 [2 rows x 112 columns])

In [4]:
df.head(), df.tail()

(   year pct ser_num datestop timestop recstat inout trhsloc perobs  \
 0  2016  41      22  2072016      100       A     O       P   1.00   
 1  2016  10      22  2182016       30       1     O       P   8.00   
 2  2016  66       1  1012016       30       1     I       P   2.00   
 3  2016  47      18  1012016       40       1     O       H   1.00   
 4  2016  79       1  1012016       50       1     O       P   3.00   
 
       crimsusp  ... zip addrpct sector beat post   xcoord   ycoord dettypCM  \
 0         BURG  ...          41      B    2       1013353   234000       CM   
 1  MISDEMEANOR  ...          10      D             983478   212373       CM   
 2          FEL  ...          66      F             988340   172111       CM   
 3          FEL  ...          47      C                                   CM   
 4       D.W.I.  ...          79      G    4        998197   187413       CM   
 
   lineCM detailCM  
 0      1       14  
 1      1       28  
 2      1        9  
 3    

In [5]:
list(df.columns) # take a look at all the columns

['year',
 'pct',
 'ser_num',
 'datestop',
 'timestop',
 'recstat',
 'inout',
 'trhsloc',
 'perobs',
 'crimsusp',
 'perstop',
 'typeofid',
 'explnstp',
 'othpers',
 'arstmade',
 'arstoffn',
 'sumissue',
 'sumoffen',
 'compyear',
 'comppct',
 'offunif',
 'officrid',
 'frisked',
 'searched',
 'contrabn',
 'adtlrept',
 'pistol',
 'riflshot',
 'asltweap',
 'knifcuti',
 'machgun',
 'othrweap',
 'pf_hands',
 'pf_wall',
 'pf_grnd',
 'pf_drwep',
 'pf_ptwep',
 'pf_baton',
 'pf_hcuff',
 'pf_pepsp',
 'pf_other',
 'radio',
 'ac_rept',
 'ac_inves',
 'rf_vcrim',
 'rf_othsw',
 'ac_proxm',
 'rf_attir',
 'cs_objcs',
 'cs_descr',
 'cs_casng',
 'cs_lkout',
 'rf_vcact',
 'cs_cloth',
 'cs_drgtr',
 'ac_evasv',
 'ac_assoc',
 'cs_furtv',
 'rf_rfcmp',
 'ac_cgdir',
 'rf_verbl',
 'cs_vcrim',
 'cs_bulge',
 'cs_other',
 'ac_incid',
 'ac_time',
 'rf_knowl',
 'ac_stsnd',
 'ac_other',
 'sb_hdobj',
 'sb_outln',
 'sb_admis',
 'sb_other',
 'repcmd',
 'revcmd',
 'rf_furt',
 'rf_bulg',
 'offverb',
 'offshld',
 'forceuse',


##### are there any columns we don't care about that can be removed?   
This dataset has granular detail on 12000+ incidents.  Let's simplify this. Taking a look at a key provided for this data, there are many detailed records for each individual stop.  The dataset provides information on if an arrest or summons was made, if a weapon was found, if force was used, demographic info about the person stopped, and location information about when and where the stop occured, as well as information regarding the officer who made the stop. Quite a few columnns, for the purpose of this investigation, can be removed.  To do so, one can use the `.drop() method`.  

In [6]:
df = df.drop(columns=['year',
 'ser_num',
 'recstat',
 'inout',
 'trhsloc',
 'perobs',
 'perstop',
 'typeofid',
 'explnstp',
 'othpers',
 'compyear',
 'comppct',
 'offunif',
 'officrid',
 'adtlrept',
 'riflshot',
 'asltweap',
 'machgun',
 'othrweap',
 'radio',
 'ac_rept',
 'ac_inves',
 'rf_vcrim',
 'rf_othsw',
 'ac_proxm',
 'rf_attir',
 'cs_objcs',
 'cs_descr',
 'cs_casng',
 'cs_lkout',
 'rf_vcact',
 'cs_cloth',
 'cs_drgtr',
 'ac_evasv',
 'ac_assoc',
 'cs_furtv',
 'rf_rfcmp',
 'ac_cgdir',
 'rf_verbl',
 'cs_vcrim',
 'cs_bulge',
 'cs_other',
 'ac_incid',
 'ac_time',
 'rf_knowl',
 'ac_stsnd',
 'ac_other',
 'sb_hdobj',
 'sb_outln',
 'sb_admis',
 'sb_other',
 'repcmd',
 'revcmd',
 'rf_furt',
 'rf_bulg',
 'offverb',
 'offshld',
 'forceuse',
 'dob',
 'ht_feet',
 'ht_inch',
 'weight',
 'haircolr',
 'eyecolor',
 'build',
 'othfeatr',
 'addrtyp',
 'rescode',
 'premtype',
 'premname',
 'addrnum',
 'stname',
 'stinter',
 'crossst',
 'aptnum',
 'state',
 'zip',
 'addrpct',
 'sector',
 'beat',
 'post',
 'dettypCM',
 'lineCM',
 'detailCM'])

In [7]:
# return a list of remaining columns 
list(df.columns)

['pct',
 'datestop',
 'timestop',
 'crimsusp',
 'arstmade',
 'arstoffn',
 'sumissue',
 'sumoffen',
 'frisked',
 'searched',
 'contrabn',
 'pistol',
 'knifcuti',
 'pf_hands',
 'pf_wall',
 'pf_grnd',
 'pf_drwep',
 'pf_ptwep',
 'pf_baton',
 'pf_hcuff',
 'pf_pepsp',
 'pf_other',
 'sex',
 'race',
 'age',
 'city',
 'xcoord',
 'ycoord']

##### lets create a few new columns. was the person innocent?  and, was the person carrying a pistol or knife?  and, was force used by the police? 

In [8]:
# Create a new column called df.crime where the value is yes
# if df.arstmade or df.sumissue are Y

#df['crime'] = np.where((df['arstmade'] == 'Y')|(df['sumissue'] == df['Y']), 'yes', 'no')

#df['weapon'] = np.where((df['pistol'] == 'Y')|(df['knifcuti'] == df['N']), 'yes', 'no')
#df['force_used'] = np.where((df['pf_hands'] == 'Y')|
                          #  (df['pf_wall'] == df['Y'])|
                          #  (df['pf_grnd'] == df['Y'])|
                          #  (df['pf_drwep'] == df['Y'])|
                          #  (df['pf_ptwep'] == df['Y'])|
                           # (df['pf_baton'] == df['Y'])|
                           # (df['pepsp'] == df['Y'])|
                           # (df['pf_other'] == df['Y']), 'yes', 'no')


##### after taking a look at the data and identifying some coordinates, further inspect the geometry columns.

These are labeled `xcoord` and `ycoord`.  Let's figure out what type of data this is using `type()` along with bracket notation to drill into specific cells within columns. 


In [9]:
# check datatype of 'xcoord' & 'ycoord'
type(df.xcoord[1]), type(df.ycoord[1]) # str

(str, str)

##### since these are strings, removing extra spaces at the beginning or end of the string that may be lingering is necessary before converting over to a numerical type
This is done with using the `.map()` method (which allows for changes to an entire series), with the `.str.strip()`.  This was completed for a variety of columns that utilized strings.  

In [10]:
#remove extra spaces in strings
df["xcoord"] = df["xcoord"].map(str.strip)  
df["ycoord"] = df["ycoord"].map(str.strip)
df["datestop"] = df["datestop"].map(str.strip)
df["timestop"] = df["timestop"].map(str.strip)
df["pct"] = df["pct"].map(str.strip)

In [11]:
# lets look at the first 5 rows of data in this column
df.xcoord.head(), df.ycoord.head()  # and we see the 4th row has no coordinate in either

(0    1013353
 1     983478
 2     988340
 3           
 4     998197
 Name: xcoord, dtype: object, 0    234000
 1    212373
 2    172111
 3          
 4    187413
 Name: ycoord, dtype: object)

##### Finding an empty row  in the first 5 of 12405 rows is _sheer luck_. If the first 5 rows were geocoded, that error would not have been observed. How could we search for these? 

Since these are `str`, they won't have any `'NaN'` values.  Rather than visually parse this for bad data, lets assume that any cells without coordinates have some other value within them (or are empty strings, especially since we stripped zeros).  If we sort for the most frequent value in this column, it might return a default string value when there is no location information.

In [12]:
# check to see what the most common value
# .idxmax() returns the most common value in a column
df.xcoord.value_counts().idxmax(), df.ycoord.value_counts().idxmax()  # answer will an empty string value ('')

('', '')

##### after identifying the cell value that indicates ***not geocoded***,  now remove ungeocoded cells from the dataframe
This will occur in 2 phases.  In the first,  replace all empty strings `''`, with `'NaN'`, and then check the datashape for comparison after phase 2.  In phase 2, we will drop all rows with `'NaN'` values in the columns specified, and then check the shape to verify that rows were dropped. 

In [13]:
# replace empty strings with NaN
df['xcoord'].replace('', np.nan)
df['ycoord'].replace('', np.nan)
df.shape # check number of rows here for comparison with result of next step


(12405, 28)

In [14]:
# Now drop the null values
df.dropna(subset=['xcoord'])
df.dropna(subset=['ycoord'])
df.shape # check number of rows here for comparison with result of previous step to verify removal

(12405, 28)

#### # to be useful later, we need numerical values (for now, integers), not Strings. 
Converting the coordinates into numerical values is paramount. 

In [15]:
# convert strings to integers
df.xcoord = pd.to_numeric(df.xcoord, errors='ignore')
df.ycoord = pd.to_numeric(df.ycoord, errors='ignore')
type(df.xcoord[1])

numpy.float64

##### After deleting some rows, resetting the index is important.  
But before that happens, reordering chronologically might be useful.  First, convert strings to integers, then sort in chronological order, then reset the index.  

In [17]:
# convert date strings to integers  & sort chronologically
#df.datestop.replace('', np.nan)
#df.dropna(subset=['datestop'])
df.datestop = df['datestop'].apply(int)
df.sort_values(by=['datestop'],inplace = True) # sort dataframe by 'date', will sort smallest to largest value, so january 1 will be first! 

ValueError: invalid literal for int() with base 10: ''

In [ ]:
# reset index
df = df.reset_index()

# delete prior index
del df['index'] 
print(df.datestop.head())  #note these first 5 values are all on jan 1 2016, verifying it worked! 

#### Lets drill into the dataset a bit with some queries.

##### which precinct had the most number of stop and frisks in 2016?  Let's figure that out and end up with a dataframe for merging with our Precincts GeoJson

In [ ]:
# extract series from df 
counts = df['pct'].value_counts()
counts.head()

In [ ]:
# convert series to pandas dataframe & reset index
counts = pd.DataFrame(counts).reset_index()
type(counts), counts.shape, counts.columns, counts.head()

In [ ]:
# rename columns 
counts.rename(columns={'index': 'precinct', 'pct': 'counts'}, inplace=True)
counts.head(), type(counts)

###### we'll park this here and come back when we import the NYPD Precincts geojson 

##### now, find all instances with 'use of force'

In [ ]:
### find total number of instances of use of force used
a = df.loc[(df['pf_pepsp'] == 'Y') | (df['pf_hands'] == 'Y') | (df['pf_wall'] == 'Y') | (df['pf_grnd'] == 'Y') | (df['pf_drwep'] == 'Y') | (df['pf_ptwep'] == 'Y') | (df['pf_baton'] == 'Y') | (df['pf_hcuff'] == 'Y') | (df['pf_other'] == 'Y')]
count = str(a.shape[0])
print('There were ' + count + ' uses of force during SQF in 2016.')

#### find all instances of minors in the Bronx caught with pistols

In [ ]:
df['pistol'].unique() # verify that values are present
a = df.loc[df['pistol'].isin(['Y']) & (df['city'] == 'BRONX') & (df['age'] < '18')] # query for various conditions
print(a)

In [ ]:
# stash the data locally as json
with open('./data/2016stops.json', 'w') as f:
    f.write(df.to_json())

##### now verify the file was written 

In [ ]:
# imports the OS module, using the .walk() method returns information about the current working directory. 
import os
for root, dirs, files in os.walk('./data/'):
    for filename in files:
        print(filename)

#### so, we've got a JSON.  To continue, we'll need to define geometry for this using Shapely, prior to converting into a GeoDataFrame for all our awesome web-mapping applications downstream.  

##### Importing geopandas and shapely modules will allow convert coordinates into points with a projection.  

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, mapping

##### access JSON that we created and stashed locally and load as dataframe

In [ ]:
path_to_file = './data/2016stops.json'
df = pd.read_json(path_to_file) # load csv data as pandas DataFrame
type(df), df.head()

##### create the geometry...
this requires us to know the projection that the `xcoord` and `ycoord` were presented.  after a bit of digging, I was able to determine that NYPD used ESPG:2263 as the projected coordinate system.  

In [ ]:
# create geometry using Shapely Point & assign proper projection
geometry = [Point(xy) for xy in zip(df.xcoord, df.ycoord)]
df = df.drop(['xcoord', 'ycoord'], axis=1)
crs = {'init': 'epsg:2263'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
gdf.head()

##### reproject data to web-map ready EPSG:4326

In [ ]:
gdf.to_crs(epsg=4326, inplace=True)
#gdf.to_crs({'init': 'epsg:4269'})
gdf.crs

##### visualize the data (are we mapping yet?), but first we'll need to import some plotting tools. 

In [ ]:
# import plotter 
%matplotlib inline

import matplotlib.pyplot as plt
# change default figsize
plt.rcParams['figure.figsize'] = (30, 24)

In [ ]:
fig, ax = plt.subplots()
 
base_color = '#f0f0f0'
border_color = base_color
 
gdf.plot(ax=ax, color='red', alpha=0.1, zorder=0, label='stops');  #low opacity adds a heatmap effect at this scale.  unintentional
 
ax.set(title="stop and frisks 2016",
       xlabel="X coordinates",
       ylabel="Y coordinates") 
ax.legend();

#### its alive!  bring in some other data to see if everything appears to be in order

In [ ]:
# import datasets
pr = gpd.read_file('./data/nypd-precincts.geojson') # import precincts geometry

##### first double-check the CRS on the data

In [ ]:
# check CRS
pr.crs

In [ ]:
# let's visualize these new layers
fig, ax = plt.subplots()
 
base_color = 'white'
border_color = base_color

# layers to print
gdf.plot(ax=ax, color='red', alpha=0.04, zorder=3, label='stops');  #low opacity adds a heatmap effect at this scale.  unintentional
pr.plot(ax=ax, edgecolor='black', color='white', alpha=1, legend = True, zorder=2);

# add title and labels
ax.set(title="stop and frisks by Precinct",
       xlabel="X coordinates",
       ylabel="Y coordinates") 

# reset bounds to match NYC data
ax.set(xlim=(-74.3,-73.6), ylim=(40.25,41))

##### let's merge the precinct "counts" into the precincts geojson

In [ ]:
# check the columns
pr.columns

In [ ]:
# first check data types to make sure they match up
type(counts.precinct[0]), type(pr.precinct[0])

In [ ]:
# convert integer to string
pr.precinct = pr.precinct.astype(str)

# first check data types to make sure they match up
type(counts.precinct[0]), type(pr.precinct[0])

In [ ]:
# Merge with `merge` method on shared variable (precinct):
pr = pr.merge(counts, on='precinct')
print(pr)

##### Now we can visualize the Precincts geojson, as a chloropeth, with the counts of Stop and Frisk Incidents per precinct as the ramp value. 
this uses matplotlib's colormap parameter `cmap`

In [ ]:
fig, ax = plt.subplots()
 
base_color = 'white'
border_color = base_color

# layers to print
pr.plot(ax=ax, edgecolor='black', column='counts', cmap = 'RdPu', alpha=1, legend = True, zorder=2);
gdf.plot(ax=ax, markersize=1, alpha=1, color='black', zorder=3);  #low opacity adds a heatmap effect at this scale.

# add title and labels
ax.set(title="stop and frisks by Precinct",
       xlabel="X coordinates",
       ylabel="Y coordinates") 

# reset bounds to match NYC data
ax.set(xlim=(-74.3,-73.6), ylim=(40.25,41))

quickly take a look at `pr` Precincts

In [ ]:
#stacking these data inspection commands is possible using commas
pr.sort_values(by=['precinct'],inplace = True),pr.head(),type(pr.precinct[0]),pr.precinct[0]

In [ ]:
type(pr.precinct[0])

#### Let's look at the busiest precinct.  If you remember from earlier, that was 106. 

In [ ]:
# clip the precinct 106
pr106 = pr.loc[pr['precinct'] == '106']

In [ ]:
# check geometry
pr106.geom_type.unique()

In [ ]:
# check bounding box of precinct 106
pr106.bounds

In [ ]:
# Plot these using the following technique, creating subplots assigned to the fig and ax variables, 
# setting a couple default variables for the colors, 
# and plotting a background using the bounding box and the world countries from Natural Earth.
fig, ax = plt.subplots()
 
base_color = '#f0f0f0'
border_color = base_color
 
pr106.plot(ax=ax, color='white', edgecolor='black', zorder=0)
gdf.plot(ax=ax, color='black', alpha=0.1, zorder=6);


ax.set(title="precinct 106 Stop and Frisks",
       xlabel="Longitude",
       ylabel="Latitude")

# reset bounds to match precinct 106 bounds
ax.set(xlim=(-73.86,-73.8), ylim=(40.64,40.69))

##### export the stop/frisk data as webmap ready GEOJSON

In [ ]:
#export as geojson 
### throws an error unless file is deleted prior
gdf.to_file(r'./data/2016stops.geojson', driver="GeoJSON")

In [ ]:
#double check the data, for ASMR purposes alone
for root, dirs, files in os.walk('./data/'):
    for filename in files:
        print(filename)

#### Finally! export the notebook as an HTML file for easy webviewing

In [18]:
!jupyter nbconvert --to html rakatk0-python-pandas-geopandas-101.ipynb

Traceback (most recent call last):
  File "C:\Users\rkatko0001\AppData\Local\Continuum\anaconda3\envs\module-06\Scripts\jupyter-nbconvert-script.py", line 9, in <module>
    sys.exit(main())
  File "C:\Users\rkatko0001\AppData\Local\Continuum\anaconda3\envs\module-06\lib\site-packages\jupyter_core\application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "C:\Users\rkatko0001\AppData\Local\Continuum\anaconda3\envs\module-06\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\rkatko0001\AppData\Local\Continuum\anaconda3\envs\module-06\lib\site-packages\nbconvert\nbconvertapp.py", line 338, in start
    self.convert_notebooks()
  File "C:\Users\rkatko0001\AppData\Local\Continuum\anaconda3\envs\module-06\lib\site-packages\nbconvert\nbconvertapp.py", line 498, in convert_notebooks
    self.exporter = cls(config=self.config)
  File "C:\Users\rkatko0001\AppData\Local